The primary purpose of this notebook is to compute the FIM for each configuration with respect to the sources locations.
A configuration is defined by the source range and receiver depth.
This is equivalent to specifying the location (range and depth) of the receiver from the source.

**Disclaimer:** This calculation requires `uwlib` and `fimpack`, which are proprietary modules developed at BYU.

I will try to make this notebook accessible for those who don't have access to these packages, by exporting what I calculated and include them in this repo.
However, the calculation to generate the FIM data will also be included here.

In [ ]:
from pathlib import Path
import os
import tarfile
import itertools

import numpy as np
import numdifftools as nd

# import uwlib
# from fimpack import (
#     orca_manager,
#     param_conditioning,
#     matrix_manager,
#     derivative_analysis,
# )
from information_matching.utils import set_directory

In [ ]:
# Setting up directories
WORK_DIR = Path().absolute()
DATA_DIR = WORK_DIR / "data"
SVP_DIR = DATA_DIR / "svp"

In [ ]:
# To help those who don't have access to uwlib and fimpack, let's try to get the
# FIM with respect to the sourceal parameters from the tarball that I generated.
FIM_DIR = DATA_DIR / "FIMs" / "source"
tarball = FIM_DIR.parent / "fim_source.tar.gz"

if not FIM_DIR.exists():
    # Check if tarball exists. If it does, then extract.
    if tarball.exists():
        os.chdir(FIM_DIR.parent)  # Entering the FIM directory
        # Extracting the tarball
        with tarfile.open(tarball.name) as f:
            f.extractall()
        os.chdir(WORK_DIR)  # Exiting FIM directory, go back to the previous path
    else:
        # There is no FIM data. This is mainly for me to generate the data.
        FIM_DIR.mkdir()

In [ ]:
# List of TOML files
opt_toml_file = DATA_DIR / "FIM_opt.toml"
# Sound profile files
sediment_type_list = ["mud", "clay", "silt", "sand", "gravel"]
svp_toml_path_dict = {
    sed: SVP_DIR / f"svp_{sed}_35m_unit_test.toml" for sed in sediment_type_list
}
# List of sound source frequencies
freq_list = [50, 100, 200, 400]  # In Hz

The receiver depth values are the same as what's used in Michael's paper.
The source (or receiver) range is set up equally spaced from 1 km to 5 km, every 200 m.
Note that I just chose these numbers arbitrarily.
In Michael's paper, he only used 1 source range value, that is 3 km.

In [ ]:
# Source depth, receiver depth, and receiver range
source_range = np.linspace(1, 5, 21)
source_depth = np.array([8, 16])
receiver_depth = np.arange(5, 76, 5)

# Compute the FIMs

To compute the Jacobian with respect to source depth and range, we will use numerical derivative via `numdifftools`.
We will use the following functions with `numdifftools`.

In [ ]:
def TL_wrapper(source_depth, source_range, receiver_depth, orca, freq, flatten=False):
    """This is a wrapper function that will be used to define other function to
    compute transmission loss with respect to source depth, source range, and receiver
    depth later.
    """
    if isinstance(source_depth, (int, float, np.int64, np.float64)):
        # Convert the source depth into a list
        source_depth = np.array([source_depth])
    if isinstance(source_range, (int, float, np.int64, np.float64)):
        # Convert the source depth into a list
        source_range = np.array([source_range])
    if isinstance(receiver_depth, (int, float, np.int64, np.float64)):
        # Convert the source depth into a list
        receiver_depth = np.array([receiver_depth])

    # Compute the transmission loss
    TL, _, _ = uwlib.tl.calc_tl_from_orca(
        orca,
        freq,
        source_range,
        source_depth,
        receiver_depth,
        return_gf=True,
        return_nmode=True,
    )
    if flatten:
        return TL.flatten()
    else:
        return TL


def TL_given_source_depth(
    source_depth, source_range, receiver_depth, orca, freq, flatten=False
):
    return TL_wrapper(source_depth, source_range, receiver_depth, orca, freq, flatten)


def TL_given_source_range(
    source_range, source_depth, receiver_depth, orca, freq, flatten=False
):
    return TL_wrapper(source_depth, source_range, receiver_depth, orca, freq, flatten)


def TL_given_receiver_depth(
    receiver_depth, source_range, source_depth, orca, freq, flatten=False
):
    return TL_wrapper(source_depth, source_range, receiver_depth, orca, freq, flatten)

For each receiver, the Jacobian should look like the following:

| ______ | depth source 1 | range source 1 | depth source 2 | range source 2 |
| :-------- | :--------------: | :--------------: | :--------------: | :--------------: |
| source 1 | $d_1$ | $d_2$ | 0 | 0 |
| source 2 | 0 | 0 | $d_3$ | $d_4$ |

Explanation: </br>
Note that we don't consider any interference between the sound from the two sources.
Thus, the measured TL of source 1 should be independent of the location of source 2, and vice versa.

How to obtain the non-zero elements: </br>
Suppose we want to get the Jacobian corresponding to receiver with depth index $i$ and range index $j$.
Then, $d_1$ and $d_3$ should be obtained from the diagonal element of `J_sd_reshaped[:, i, j]`, where `J_sd` is the Jacobian with respect to source depth.
$d_2$ and $d_4$ are extracted from `J_sr_reshaped[:, i, j, j]`, where `J_sr` is the Jacobian with respect to source range.

In [ ]:
# We will use the following tuples to indicate the configurations
configs = list(itertools.product(source_range, receiver_depth))
print("Number of configurations:", len(configs))

From previous testing, we will use the following settings in `numdifftools`:
* Use `order=4`
* For the step size for `d1` and `d3` (correspond to the depth, which is measured in m), set it to `1e-1`, or 0.1 m
* For the step size for `d2` and `d4` (correspond to the range, which is measured in km), set it to `5e-3`, or 5 m.

In [ ]:
# %%capture

for sediment_type in sediment_type_list:
    # Iterate over the sediment layer types
    print("Sediment:", sediment_type)

    svp_toml_file = svp_toml_path_dict[sediment_type]
    FIM_SED_DIR = set_directory(FIM_DIR / sediment_type)

    for freq in freq_list:
        # Iterate over the frequencies
        print("Frequency:", freq, "Hz")

        FIM_SED_FREQ_DIR = set_directory(FIM_SED_DIR / f"f{int(freq)}Hz")
        jac_sed_freq_file = FIM_SED_FREQ_DIR / "jacobian_all_configs.npz"
        print(sediment_type, freq)
        if jac_sed_freq_file.exists():
            jac_sed_freq = np.load(jac_sed_freq_file)
            J_sd_reshaped = jac_sed_freq["J_source_depth"]
            J_sr_reshaped = jac_sed_freq["J_source_range"]
        else:
            orca = orca_manager.initialize_orca(
                freq, str(svp_toml_file), str(opt_toml_file)
            )

            # Compute the Jacobian
            J_sd = nd.Jacobian(TL_given_source_depth, step=0.1, order=4)(
                source_depth, source_range, receiver_depth, orca, freq, flatten=True
            )
            J_sr = nd.Jacobian(TL_given_source_range, step=5e-3, order=4)(
                source_range, source_depth, receiver_depth, orca, freq, flatten=True
            )

            # Reshape the Jacobian
            J_sd_reshaped = J_sd.reshape(
                len(source_depth),
                len(receiver_depth),
                len(source_range),
                len(source_depth),
            )
            J_sr_reshaped = J_sr.reshape(
                len(source_depth),
                len(receiver_depth),
                len(source_range),
                len(source_range),
            )
            np.savez(
                jac_sed_freq_file,
                J_source_depth=J_sd_reshaped,
                J_source_range=J_sr_reshaped,
            )

        # Get the Jacobian for each configuration
        for ii, conf in enumerate(configs):
            fim_file_path = FIM_SED_FREQ_DIR / f"config_{ii}.npz"
            # Elements of the Jacobian
            r, d = conf
            i = np.where(receiver_depth == d)[0][0]  # Receiver depth index
            j = np.where(source_range == r)[0][0]  # Receiver range index
            d1, d3 = np.diag(J_sd_reshaped[:, i, j])
            d2, d4 = J_sr_reshaped[:, i, j, j]
            # Put them together
            jacobian = np.array([[d1, d2, 0, 0], [0, 0, d3, d4]])
            fim = jacobian.T @ jacobian

            # Export
            np.savez(fim_file_path, jacobian=jacobian, fim=fim)

In [ ]:
# # Create a tarball
# if not tarball.exists():
#     os.chdir(FIM_DIR.parent)  # Entering the FIM directory
#     # Creating a tarball
#     with tarfile.open(tarball.name, "w:gz") as tar:
#         tar.add(FIM_DIR.name)
#     os.chdir(WORK_DIR)  # Exiting FIM directory, go back to the previous path